# Dependencies

In [15]:
import pandas as pd

# Load the Data

In [16]:
fremont_bridge = 'https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD'

bicycle_weather = 'https://raw.githubusercontent.com/jakevdp/PythonDataScienceHandbook/master/notebooks/data/BicycleWeather.csv'

counts = pd.read_csv(fremont_bridge, index_col='Date', parse_dates=True, 
                     infer_datetime_format=True)

weather = pd.read_csv(bicycle_weather, index_col='DATE', parse_dates=True, 
                      infer_datetime_format=True)

daily = counts.resample('d').sum()
daily['Total'] = daily.sum(axis=1)
daily = daily[['Total']] # remove other columns

weather_columns = ['PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'AWND']
daily = daily.join(weather[weather_columns], how='inner')

# Make a feature for yesterday's total
daily['Total_yesterday'] = daily.Total.shift(1)
daily = daily.drop(index=daily.index[0])

daily.head()

,Total,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,Total_yesterday
2012-10-04,3475.0,0,0,0,189,83,65,3521.0
2012-10-05,3148.0,0,0,0,217,89,57,3475.0
2012-10-06,2006.0,0,0,0,239,78,51,3148.0
2012-10-07,2142.0,0,0,0,239,78,13,2006.0
2012-10-08,3537.0,0,0,0,211,78,19,2142.0


# Create Test/Train Sets

In [17]:
X_train = daily.drop(columns='Total')[:-100]
X_test = daily.drop(columns='Total')[-100:]

y_train = daily['Total'][:-100]
y_test = daily['Total'][-100:]

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((963, 7), (100, 7), (963,), (100,))

# Detect & Replace Incorrect  Data

In [19]:
daily.describe()

,Total,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,Total_yesterday
count,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000
mean,2632.449671,29.350894,-37.496707,0.098777,166.863594,84.472248,22.338664,2633.056444
std,1252.864020,65.813053,612.512583,2.570041,74.779734,50.916006,307.984292,1253.138245
min,98.000000,0.000000,-9999.000000,0.000000,-16.000000,-71.000000,-9999.000000,98.000000
25%,1806.000000,0.000000,0.000000,0.000000,111.000000,44.000000,22.000000,1806.000000
50%,2435.000000,0.000000,0.000000,0.000000,150.000000,83.000000,29.000000,2435.000000
75%,3574.500000,26.500000,0.000000,0.000000,222.000000,128.000000,40.000000,3574.500000
max,6088.000000,559.000000,74.000000,80.000000,356.000000,183.000000,95.000000,6088.000000


In [20]:
def clean(X):
    AWND_mean = X_train['AWND'].mean()
    SNOW_mean = X_train['SNOW'].mean()

    X['AWND'] = X['AWND'].replace({-9999:AWND_mean})
    X['SNOW'] = X['SNOW'].replace({-9999:SNOW_mean})
    
    return X

In [21]:
X_train = clean(X_train)
X_test = clean(X_test)

# Create Features

In [22]:
import numpy as np

In [23]:
def make_features(X):
    X = X.copy()

    # patterns of use generally vary from day to day; 
    # let's add binary columns that indicate the day of the week:
    days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    for i, day in enumerate(days):
        X[day] = (X.index.dayofweek == i).astype(float)


    # we might expect riders to behave differently on holidays; 
    # let's add an indicator of this as well:
    from pandas.tseries.holiday import USFederalHolidayCalendar
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays('2012', '2016')
    X = X.join(pd.Series(1, index=holidays, name='holiday'))
    X['holiday'].fillna(0, inplace=True)
    
    # We also might suspect that the hours of daylight would affect 
    # how many people ride; let's use the standard astronomical calculation 
    # to add this information:
    def hours_of_daylight(date, axis=23.44, latitude=47.61):
        """Compute the hours of daylight for the given date"""
        days = (date - pd.datetime(2000, 12, 21)).days
        m = (1. - np.tan(np.radians(latitude))
             * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
        return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

    X['daylight_hrs'] = list(map(hours_of_daylight, X.index))
    
    # temperatures are in 1/10 deg C; convert to C
    X['TMIN'] /= 10
    X['TMAX'] /= 10
    
    # We can also calcuate the average temperature.
    X['Temp (C)'] = 0.5 * (X['TMIN'] + X['TMAX'])
    
    # Convert Temperatures to Farenheit
    X['TMIN'] = X['TMIN'] * 1.8 + 32
    X['TMAX'] = X['TMAX'] * 1.8 + 32
    X['Temp (C)'] = X['Temp (C)'] * 1.8 + 32
    X = X.rename(columns={'Temp (C)':'Temp (F)'})
    
    # precip is in 1/10 mm; convert to inches
    X['PRCP'] /= 254

    # In addition to the inches of precipitation, let's add a flag that 
    # indicates whether a day is dry (has zero precipitation):
    X['dry day'] = (X['PRCP'] == 0).astype(int)

    # Let's add a counter that increases from day 1, and measures how many 
    # years have passed. This will let us measure any observed annual increase 
    # or decrease in daily crossings:
    X['annual'] = (X.index - X.index[0]).days / 365.
    
    # Create feature to indicate how many standardeviations from the mean the data is 
    temp_mean = X['Temp (F)'].mean()
    temp_std = X['Temp (F)'].std()
    X['Temp_STD_from_mean'] = abs(X['Temp (F)'] - temp_mean) / temp_std
    
    # Create feature that record's the temperate from yesterday
    X['Yesterdays Temp'] = X.tshift(periods=1)['Temp (F)'] 
    X['Yesterdays Temp'] = X['Yesterdays Temp'].fillna(method='bfill')
    
    # Create feature that records the total number of crossings from a week ago
    X['Total_yesterday'] = X.shift(periods=7)['Total_yesterday']
    X['Total_yesterday'] = X['Total_yesterday'].fillna(method='bfill')
    
    # DS1 DH
    X['PRCP_yesterday'] = X.PRCP.shift(1).fillna(X.PRCP.mean())
    X['Windchill'] = (((X['Temp (F)'] * (9/5) + 32) * .6215) + 34.74) - (35.75 * (X['AWND']** .16)) + (.4275 * (X['Temp (F)'])) * (X['AWND'] ** .16)
    X['Rl_Cold'] = (((X['Temp (F)'] * (9/5) + 32) - X['Windchill']) -32) * (5/9)
    X['TMIN_squared'] = X['TMIN'] **2
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i, month in enumerate(months):
        X[month] = (X.index.month == i+1).astype(float)
    
    # DS3 JD
    X['light_rain'] = (X['PRCP'] > 0) & (X['PRCP'] < 0.10)
    X['moderate_rain'] = (X['PRCP'] >= 0.1) & (X['PRCP'] < 0.30)
    X['heavy_rain'] = (X['PRCP'] >= 0.30)
    X['weekend_day'] = (X['Sat'] == 1) | (X['Sun'] == 1)
    
    return X

In [24]:
X_train = make_features(X_train)
X_test = make_features(X_test)

# Preprocessing

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

In [41]:
cont_cols = ['PRCP', 'SNWD', 'TMAX', 'TMIN', 'AWND', 'Total_yesterday', 'daylight_hrs', 'Temp (F)', 'annual', 'Temp_STD_from_mean', 'Yesterdays Temp']
preprocess = make_column_transformer(
    (StandardScaler(), cont_cols)
)

# XGBoost Modeling

In [42]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [47]:
from xgboost import XGBRegressor

build_boost = Pipeline(steps=[
        ('model', XGBRegressor(n_jobs=-1))
])

param_grid = {
    'model__n_estimators':[100, 125, 150, 250],
    'model__learning_rate':[.04, .05, .06],
    'model__max_depth':[5, 6, 7]
}

In [48]:
from sklearn.model_selection import GridSearchCV

search = GridSearchCV(build_boost, 
                      param_grid=param_grid, 
                      return_train_score=True, 
                      scoring='neg_mean_absolute_error', 
                      cv=3,
                      n_jobs=-1)
search.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('model', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'model__n_estimators': [100, 125, 150, 250], 'model__learning_rate': [0.04, 0.05, 0.06], 'model__max_depth': [5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_absolute_error', verbose=0)

In [49]:
print('Best Score:', search.best_score_)
print('Best Params:', search.best_params_)

Best Score: -291.28709974120585
Best Params: {'model__learning_rate': 0.04, 'model__max_depth': 5, 'model__n_estimators': 250}


In [50]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__learning_rate,param_model__max_depth,param_model__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
3,1.196210,0.010767,0.009908,0.000448,0.04,5,250,"{'model__learning_rate': 0.04, 'model__max_dep...",-287.049677,-304.859678,-281.951944,-291.287100,9.820315,1,-70.680155,-70.904577,-71.617907,-71.067546,0.399803
2,0.725579,0.012758,0.007034,0.000302,0.04,5,150,"{'model__learning_rate': 0.04, 'model__max_dep...",-282.926755,-307.632552,-286.209392,-292.256233,10.954978,2,-101.643501,-96.254972,-98.833348,-98.910607,2.200536
1,0.609074,0.025311,0.006511,0.000198,0.04,5,125,"{'model__learning_rate': 0.04, 'model__max_dep...",-276.345727,-315.098382,-286.310447,-292.584852,16.431031,3,-113.759979,-108.629875,-112.815375,-111.735077,2.229316
16,0.624397,0.052786,0.010526,0.004732,0.05,6,100,"{'model__learning_rate': 0.05, 'model__max_dep...",-272.937854,-313.292727,-292.181383,-292.803988,16.480689,4,-82.396344,-77.062454,-78.991447,-79.483415,2.205164
17,0.875598,0.034120,0.020348,0.014442,0.05,6,125,"{'model__learning_rate': 0.05, 'model__max_dep...",-282.428511,-306.077799,-292.214122,-293.573477,9.702512,5,-65.164959,-63.786570,-61.987985,-63.646505,1.300770
12,0.490528,0.007414,0.008120,0.002829,0.05,5,100,"{'model__learning_rate': 0.05, 'model__max_dep...",-275.858107,-314.810933,-290.079069,-293.582703,16.094249,6,-113.830659,-108.315825,-114.410626,-112.185704,2.746641
13,0.609491,0.011470,0.006885,0.000248,0.05,5,125,"{'model__learning_rate': 0.05, 'model__max_dep...",-282.521621,-307.551070,-292.463451,-294.178714,10.289960,7,-98.939241,-93.283794,-98.709932,-96.977656,2.613632
15,1.220476,0.033993,0.013934,0.002417,0.05,5,250,"{'model__learning_rate': 0.05, 'model__max_dep...",-287.291263,-308.845733,-286.936088,-294.357695,10.245616,8,-57.114979,-61.566700,-62.394590,-60.358757,2.318465
14,0.724368,0.003926,0.007731,0.000602,0.05,5,150,"{'model__learning_rate': 0.05, 'model__max_dep...",-285.029646,-306.188680,-292.866001,-294.694776,8.734395,9,-88.729764,-84.572143,-89.119254,-87.473720,2.057878
18,0.927429,0.048259,0.012950,0.003795,0.05,6,150,"{'model__learning_rate': 0.05, 'model__max_dep...",-285.996297,-306.326635,-292.301772,-294.874901,8.496918,10,-54.811136,-54.841162,-53.833561,-54.495287,0.468071


In [53]:
from sklearn.metrics import mean_absolute_error

final = search.best_estimator_
y_pred = final.predict(X_test)

test_mae = mean_absolute_error(y_test, y_pred)
print('Mean Absolute Error, with final model, on the hold-out test set ', test_mae)

Mean Absolute Error, with final model, on the hold-out test set  243.40218139648437
